<a href="https://www.kaggle.com/code/aisuko/rag-with-ragsequenceforgeneration?scriptVersionId=166452644" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Before we got our own fine-tuned LLMs. How we can make our basline LLM in a higher accuracy? One of the answers might be **Retrieval Augmented Generation(RAG)**. RAG enables LLMs reach beyond their static knowledge and grab real-time information quite easily. RAG taps into external sources, like internal company documents, to understand the context of our queries a bit better.

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/064/408/832/155/132/small/3f7a8bd57563298f.webp)

In this notebook, we are going to use RAG model. Retrieval-agumented generation("RAG") models combine the powers of pretrained dense retrieval(DPR) and Seq2Seq2 models. RAG models retrieve docs, pass them to a seq2seq2 model, then marginalize to generate outputs. The retriever and seq2seq modules are initlized from pretrained models, and fine-tuned jointly, allowing both retreival and generation to adapt to downstream tasks. Here are some classes inside transformers that we need to know:


# RagConfig Class

It stores the configuration of a RagModel which is used to control thr model outputs.


# Retriever Class

It used to get documents from vector queries. It retrives the documents embeddings as well as the documents contents, and it formats them to be used with a RagModel.


# RAGModel Class

RAG is a seq2seq model which encapsulates two core components: a question encoder and a generator. During a forward pass, we encode the input with the question encoder and pass it to the retriever to extract releveant context documents. The documents are then prepended to the input. Such contextualized inputs is passed to the generator.

The question encoder can be any autoencoding model, preferably **DPPQuestionEncoder** and the generator can be any **seq2seq**, preferably **BartForConditionalGeneration**. So, we can initialzed with the RAG model with any autoencoding mdoel as the question_encoder and any seq2seq model with language model heads as the generator. It inherits from **PreTrainedModel**, and it is also a PyTorch **torch.nn.Module** subclass. 

In [1]:
%%capture
!conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

In [2]:
%%capture
!pip install transformers==4.38.2
# !pip install accelerate==0.27.2
!pip install datasets==2.18.0 # Fix issue numpy attributes error of RagRetriever
# !pip install peft==0.9.0
# !pip install bitsandbytes==0.42.0

In [3]:
!transformers-cli env

2024-03-11 11:03:33.723107: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 11:03:33.723199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 11:03:33.861435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.38.2
- Platform: Linux-5.15.133+-x86_64-with-glibc2.31
- Python version: 3.10.13
- Huggingface_hub version: 0.20.3
- Safetensors version: 0.4.2
- Accelerate version: 0.27.2
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.1.2 (True)
- Tens

In [4]:
import os
import torch
import warnings
# for checking faiss-gpu
import faiss

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))


os.environ["MODEL_NAME"] = "facebook/rag-token-base"
os.environ["DATASET"] = ""

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic=True
    # https://github.com/huggingface/transformers/issues/28731
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    device='cuda'
else:
    device='cpu'
    
warnings.filterwarnings('ignore')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))

config.json:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


(…)enerator_tokenizer/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [6]:
from transformers import RagRetriever

retriever=RagRetriever.from_pretrained(
    os.getenv("MODEL_NAME"), 
    index_name="exact", 
    use_dummy_dataset=True,
    trust_remote_code=True
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The to

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
from transformers import RagSequenceForGeneration

model=RagSequenceForGeneration.from_pretrained(os.getenv("MODEL_NAME"), retriever=retriever, device=device)
type(model)

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

transformers.models.rag.modeling_rag.RagSequenceForGeneration

In [8]:
input_ids=tokenizer("How many people live in Melbourne?", return_tensors="pt")

In [9]:
generated=model.generate(input_ids["input_ids"], max_length=50)
generated_str=tokenizer.decode(generated[0], skip_special_tokens=True)
generated_str

'The!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

# Reference List

* https://www.promptingguide.ai/research/rag
* https://blog.gopenai.com/retrieval-augmented-generation-rag-585aa903d6bd
* https://github.com/ray-project/ray
* https://github.com/facebookresearch/faiss/blob/main/INSTALL.md
* https://lightning.ai/docs/pytorch/stable/